In [4]:
import twitter
import json
from prettytable import PrettyTable
from collections import Counter

In [5]:
##Lectura de claves de Twitter que estan definidos en un archivo JSON
##sin trackeo de git

with open('twitter_keys.json') as data_file:    
    data = json.load(data_file)

OAUTH_TOKEN = data["OAUTH_TOKEN"]
OAUTH_TOKEN_SECRET = data["OAUTH_TOKEN_SECRET"]
CONSUMER_KEY = data["CONSUMER_KEY"]
CONSUMER_SECRET = data["CONSUMER_SECRET"]

In [6]:
auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth)

In [7]:
#Se realiza la consulta a Twitter
ARG_WOE_ID = 23424747
arg_trends = twitter_api.trends.place(_id=ARG_WOE_ID)


# Dumpear los tweets a disco

In [10]:
print type(arg_trends)

<class 'twitter.api.TwitterListResponse'>


In [13]:
#Dumpear a disco
with open("tweets-json-pto1.json", "w") as outfile:
    json.dump(arg_trends, outfile, indent=5)

In [24]:
#Obtener de disco
with open('tweets-json-pto1.json', 'r') as f:
     tweets_arg = json.load(f)

print (tweets_arg)



[{u'woeid': 23424747, u'name': u'Argentina'}]
